In [ ]:
import sys
import os
sys.path.append('/app/src')
from setup_environment import setup_paths, seed_localization_and_tip_tracking
c, box, util, model = setup_paths()


In [ ]:
%%capture 
#capture prevents output of function calls from being printed (ie, lots of tensorflow verbiage)
data_path = c.QUANTIFICATION_IN_PATH
seed_model = model.SeedModel(c.SEED_MODEL_PATH)
qr_model = model.QrModel(c.QR_MODEL_PATH)

2022-07-08 03:11:51.025926: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-07-08 03:11:51.036608: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200155000 Hz
2022-07-08 03:11:51.038287: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55f6701b2070 executing computations on platform Host. Devices:
2022-07-08 03:11:51.038436: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>


In [ ]:
box_list = util.listdir_nohidden(os.path.join(data_path, "stabilized"))
outputs = []
print(box_list)

#threshold_multiplier:
#1.3 and 50 can work for rice. rice has been 1.5 (bright lights) and 1.3 or 1.4 (dimmer lights) and 30-50, arabidopsis seemed to work somewhat at 1.4 and 10
#1.2 was working pretty well for rice in high gelzan percent boxes (.5% or above)
for expt in box_list:
    
    while True:
        track = input("Would you like to track box #" + str(expt) + "? (y) or (n): ")
        if track == "y" or track == "n":
            break
        else:
            print("Invalid character")
    if track == "y":
        box_path = os.path.join(data_path, "stabilized", expt, "")
        b = box.Box(box_path)
        b.init_seeds(seed_model, automatic = True) 
        b.germination_detection(save_tip_sample=False, threshold_multiplier= 1.5, automatic = False)
        b.tip_trace_pcv(384, threshold_multiplier = 1.5, bound_radius = 30) 
        b.validate_save_tracking()
        del b
        gc.collect()  

In [ ]:
util.sync_for_rstudio()
util.sync_traced_tips() #NEED TO FIGURE OUT HOW TO SYNC TO BUCKET. CAN'T SIMPLY USE RSYNC TO MOUNTED BUCKET

In [ ]:
util.archive("stabilized")

In [ ]:
#run this if the program crashes
del b
gc.collect()

In [ ]:
util.empty_trash()